This notebook combines the individual model's rating to form a unified model which performs better. The ratings from individual models are combined using a weighted linear combination to form a resultant rating. This method helps overcome the shortcomings of individual method. \\
The ratings are those generated using surprise library.

In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
pred_data = pd.read_csv('../2_results/csv/test_prediction_HP.csv')
pred_data.head() 

,Unnamed: 0,uid,iid,og_rating,svd_rating,knn_rating,svdpp_rating,slopeone_rating,baseline_rating
0,0,671,51927,4.0,3.716340,4.037208,3.892101,3.926010,3.991227
1,1,671,14291,4.0,3.995477,3.960943,4.109132,4.119416,3.986223
2,2,671,10162,4.0,3.940816,4.113167,3.856270,3.868475,4.131273
3,3,671,9479,5.0,3.757214,3.273754,3.343759,3.758168,3.569950
4,4,671,8872,3.0,3.451297,3.483119,3.516192,3.635662,3.870820


In [3]:
pred_data = pred_data.rename(columns={"knn_rating": "knnwm_rating", "baseline_rating": "knn_rating"})

In [4]:
pred_data = pred_data.drop('slopeone_rating', axis = 1)

In [5]:
pred_data

,Unnamed: 0,uid,iid,og_rating,svd_rating,knnwm_rating,svdpp_rating,knn_rating
0,0,671,51927,4.0,3.716340,4.037208,3.892101,3.991227
1,1,671,14291,4.0,3.995477,3.960943,4.109132,3.986223
2,2,671,10162,4.0,3.940816,4.113167,3.856270,4.131273
3,3,671,9479,5.0,3.757214,3.273754,3.343759,3.569950
4,4,671,8872,3.0,3.451297,3.483119,3.516192,3.870820
...,...,...,...,...,...,...,...,...
19990,19990,2,197,4.0,3.594939,3.874116,3.514705,3.700537
19991,19991,1,1598,2.0,3.083082,2.775164,2.889325,2.378814
19992,19992,1,1051,4.0,3.002330,3.090589,2.999079,2.424992
19993,19993,1,819,3.0,2.795546,2.471782,2.749813,2.220614


In [6]:
# num of rows:
T = pred_data.shape[0]
print(T)

19995


In [7]:
rmse = ((pred_data.og_rating - pred_data.knn_rating) ** 2).mean() ** .5
print(rmse)
mae = (((pred_data.og_rating - pred_data.knn_rating) ** 2) ** .5).mean()
print(mae)

0.9448177762755212
0.7159112913732618


In [8]:
rmse = ((pred_data.og_rating - pred_data.knnwm_rating) ** 2).mean() ** .5
print(rmse)
mae = (((pred_data.og_rating - pred_data.knnwm_rating) ** 2) ** .5).mean()
print(mae)

0.9096249508924891
0.6919726680943921


In [9]:
rmse = ((pred_data.og_rating - pred_data.svd_rating) ** 2).mean() ** .5
print(rmse)
mae = (((pred_data.og_rating - pred_data.svd_rating) ** 2) ** .5).mean()
print(mae)

0.89998713841803
0.6924169680333669


In [10]:
import math
from collections import defaultdict
import csv
from sklearn.metrics import ndcg_score
import time
from surprise.prediction_algorithms.predictions import Prediction
from surprise import accuracy

In [11]:
def get_top_n(predictions, n):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    org_ratings = defaultdict(list)

    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
        org_ratings[uid].append((iid, true_r))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n, org_ratings

In [12]:
def dcg_at_k(scores):
    return scores[0] + sum(sc/math.log(ind, 2) for sc, ind in zip(scores[1:], range(2, len(scores) + 1)))

def ndcg_at_k(scores):
    idcg = dcg_at_k(sorted(scores, reverse=True))
    return (dcg_at_k(scores)/idcg) if idcg > 0.0 else 0.0

In [13]:
def precision_recall_at_k(predictions, k=5, threshold=3.5):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    precision = (sum(prec for prec in precisions.values()) / len(precisions))
    recall = (sum(rec for rec in recalls.values()) / len(recalls))

    return precision, recall

In [34]:
X = pred_data[['svd_rating','svdpp_rating','knn_rating', 'knnwm_rating']].to_numpy()

In [35]:
X

array([[3.71633988, 3.89210108, 3.99122746, 4.0372083 ],
       [3.995477  , 4.109132  , 3.98622314, 3.96094259],
       [3.94081572, 3.8562699 , 4.13127254, 4.11316739],
       ...,
       [3.00233011, 2.9990794 , 2.42499247, 3.09058853],
       [2.79554571, 2.7498129 , 2.22061372, 2.47178197],
       [3.3454489 , 3.14006274, 2.5584881 , 2.96619033]])

In [36]:
y = pred_data[['og_rating']].to_numpy()

In [37]:
y

array([[4.],
       [4.],
       [4.],
       ...,
       [4.],
       [3.],
       [2.]])

In [29]:
from sklearn.linear_model import LinearRegression

In [38]:
reg = LinearRegression().fit(X, y)

In [39]:
reg.coef_

array([[0.19886785, 0.37191812, 0.25684355, 0.24861044]])

In [45]:
heso = [
    [0.05, 0.4, 0.6, 0],
    [0.4, 0.6, 0, 0],
    [0.6, 0.4, 0, 0],
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [0.19886785, 0.37191812, 0.25684355, 0.24861044]
]

In [46]:
surprise_df = pd.DataFrame(columns= ['svd_wt', 'svdpp_wt', 'knn_wt', 'knnwm_wt', 'test_rmse', 'test_mae', 'Precision', 'Recall', 'F-measure', 'NDCG'])

In [47]:
for i in heso:

    svd_wt, svdpp_wt, knn_wt, knnwm_wt = i

    sqr_sum = 0
    abs_sum = 0
    
    copy = pred_data.copy()
    
    copy['new_rating'] = svd_wt*copy['svd_rating'] + knn_wt*copy['knn_rating'] + svdpp_wt*copy['svdpp_rating'] + knnwm_wt*copy['knnwm_rating']

    test_predictions = [Prediction(uid=x, iid=y, r_ui=z, est=t, details = {}) for x, y, z, t in zip(copy['uid'], copy['iid'], copy['og_rating'], copy['new_rating'])]
   
    test_rmse = accuracy.rmse(test_predictions)
    test_mae = accuracy.mae(test_predictions)
    
    top_n, org_ratings = get_top_n(test_predictions, 5)

    precision, recall = precision_recall_at_k(test_predictions)
    
    
    f_measure = (2*precision*recall)/(precision+recall)

    ndcg_scores = dict()
    for uid, user_ratings in top_n.items():
        scores = []
        for iid, est_r in user_ratings:
            iid_found = False
            org_user_ratings = org_ratings[uid]
            for i, r in org_user_ratings:
                if iid == i:
                    scores.append(r)
                    iid_found = True
                    break
            if not iid_found:
                scores.append(0)
        ndcg_scores[uid] = ndcg_at_k(scores)
    ndcg_score = sum(ndcg for ndcg in ndcg_scores.values())/len(ndcg_scores)

    # print(svd_wt, svdpp_wt, knn_wt, knnwm_wt)
    # print(test_rmse, test_mae, precision, recall, f_measure, ndcg_score)
    
    df = pd.DataFrame([[svd_wt, svdpp_wt, knn_wt, knnwm_wt, test_rmse, test_mae, precision, recall, f_measure, ndcg_score]], 
                      columns = ['svd_wt', 'svdpp_wt', 'knn_wt', 'knnwm_wt', 'test_rmse', 'test_mae', 'Precision', 'Recall', 'F-measure', 'NDCG'])
    surprise_df = pd.concat([df, surprise_df], ignore_index=True)

RMSE: 0.9255
MAE:  0.6956
RMSE: 0.8838
MAE:  0.6789
RMSE: 0.8855
MAE:  0.6806
RMSE: 0.9000
MAE:  0.6924
RMSE: 0.8917
MAE:  0.6843
RMSE: 0.9448
MAE:  0.7159
RMSE: 0.9096
MAE:  0.6920
RMSE: 0.9265
MAE:  0.6956


In [48]:
surprise_df

,svd_wt,svdpp_wt,knn_wt,knnwm_wt,test_rmse,test_mae,Precision,Recall,F-measure,NDCG
0,0.198868,0.371918,0.256844,0.24861,0.926546,0.695587,0.813835,0.452419,0.581549,0.965452
1,0.000000,0.000000,0.000000,1,0.909625,0.691973,0.807303,0.416420,0.549433,0.959655
2,0.000000,0.000000,1.000000,0,0.944818,0.715911,0.769846,0.398295,0.524980,0.950577
3,0.000000,1.000000,0.000000,0,0.891665,0.684282,0.836165,0.408657,0.549002,0.965110
4,1.000000,0.000000,0.000000,0,0.899987,0.692417,0.817511,0.396955,0.534416,0.959601
5,0.600000,0.400000,0.000000,0,0.885512,0.680589,0.834550,0.403650,0.544123,0.963212
6,0.400000,0.600000,0.000000,0,0.883826,0.678907,0.836761,0.406923,0.547563,0.964381
7,0.050000,0.400000,0.600000,0,0.925529,0.695559,0.813189,0.439440,0.570557,0.962866


In [39]:
pred_data

,uid,iid,og_rating,svd_rating,knnwm_rating,svdpp_rating,knn_rating
0,671,51927,4.0,3.716340,4.037208,3.892101,3.991227
1,671,14291,4.0,3.995477,3.960943,4.109132,3.986223
2,671,10162,4.0,3.940816,4.113167,3.856270,4.131273
3,671,9479,5.0,3.757214,3.273754,3.343759,3.569950
4,671,8872,3.0,3.451297,3.483119,3.516192,3.870820
...,...,...,...,...,...,...,...
19990,2,197,4.0,3.594939,3.874116,3.514705,3.700537
19991,1,1598,2.0,3.083082,2.775164,2.889325,2.378814
19992,1,1051,4.0,3.002330,3.090589,2.999079,2.424992
19993,1,819,3.0,2.795546,2.471782,2.749813,2.220614
